In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

In [3]:
df = pd.read_csv("kaggle\_Iowa Liquor Sales\Iowa_Liquor_Sales.csv", encoding="latin1")
df.sample(1)

C:\Users\jorge\AppData\Local\Temp\ipykernel_7924\3609158165.py:1: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("kaggle\_Iowa Liquor Sales\Iowa_Liquor_Sales.csv", encoding="latin1")


,Invoice/Item Number,Date,Store Number,Store Name,Address,City,Zip Code,Store Location,County Number,County,...,Item Number,Item Description,Pack,Bottle Volume (ml),State Bottle Cost,State Bottle Retail,Bottles Sold,Sale (Dollars),Volume Sold (Liters),Volume Sold (Gallons)
6336436,S18767300009,05/05/2014,4104,Fareway Stores #491 / Mason City,400 NORTH DELAWARE AVE,MASON CITY,50401,400 NORTH DELAWARE AVE\nMASON CITY 50401\n(43....,17.0,Cerro Gordo,...,45887,Sailor Jerry Spiced Navy Rum,12,1000,$11.78,$17.67,12,$212.04,12.0,3.17


In [4]:
df.columns

Index(['Invoice/Item Number', 'Date', 'Store Number', 'Store Name', 'Address',
       'City', 'Zip Code', 'Store Location', 'County Number', 'County',
       'Category', 'Category Name', 'Vendor Number', 'Vendor Name',
       'Item Number', 'Item Description', 'Pack', 'Bottle Volume (ml)',
       'State Bottle Cost', 'State Bottle Retail', 'Bottles Sold',
       'Sale (Dollars)', 'Volume Sold (Liters)', 'Volume Sold (Gallons)'],
      dtype='object')

FACT VENTA

In [5]:
Fact_Venta = df[["Invoice/Item Number", "Date", "Store Number", "Item Number", "Bottles Sold"]]
Fact_Venta.sample(1)

,Invoice/Item Number,Date,Store Number,Item Number,Bottles Sold
6912036,S04681300039,03/21/2012,2566,54446,12


In [6]:
Fact_Venta.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12591077 entries, 0 to 12591076
Data columns (total 5 columns):
 #   Column               Dtype 
---  ------               ----- 
 0   Invoice/Item Number  object
 1   Date                 object
 2   Store Number         int64 
 3   Item Number          int64 
 4   Bottles Sold         int64 
dtypes: int64(3), object(2)
memory usage: 480.3+ MB


In [7]:
#Cambiar tipo de dato
Fact_Venta["Date"] = pd.to_datetime(Fact_Venta["Date"])

C:\Users\jorge\AppData\Local\Temp\ipykernel_17544\1176369374.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Fact_Venta["Date"] = pd.to_datetime(Fact_Venta["Date"])


In [8]:
#Verificar si se cambio correctamente el tipo de dato
Fact_Venta.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12591077 entries, 0 to 12591076
Data columns (total 5 columns):
 #   Column               Dtype         
---  ------               -----         
 0   Invoice/Item Number  object        
 1   Date                 datetime64[ns]
 2   Store Number         int64         
 3   Item Number          int64         
 4   Bottles Sold         int64         
dtypes: datetime64[ns](1), int64(3), object(1)
memory usage: 480.3+ MB


In [9]:
#Verificar si hay valores nulos
Fact_Venta.isna().sum()

Invoice/Item Number    0
Date                   0
Store Number           0
Item Number            0
Bottles Sold           0
dtype: int64

In [10]:
Fact_Venta.rename(columns={'Store Number': 'Id Tienda', 'Invoice/Item Number': 'Id Venta', "Date": "Fecha", "Item Number": "Id Articulo", "Bottles Sold": "Botellas Vendidas"}, inplace=True)

C:\Users\jorge\AppData\Local\Temp\ipykernel_17544\1864184128.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Fact_Venta.rename(columns={'Store Number': 'Id Tienda', 'Invoice/Item Number': 'Id Venta', "Date": "Fecha", "Item Number": "Id Articulo", "Bottles Sold": "Botellas Vendidas"}, inplace=True)


In [12]:
minimo = Fact_Venta["Fecha"].min()
maximo = Fact_Venta["Fecha"].max()
print(f"La fecha minima es {minimo} y la fecha maxima es {maximo}")

La fecha minima es 2012-01-03 00:00:00 y la fecha maxima es 2017-10-31 00:00:00


In [13]:
def extraer_trimestres_año(df, columna_fecha, fecha_inicio, fecha_fin):
    """
    Realiza una consulta sobre un DataFrame utilizando un rango de fechas con `BETWEEN`.

    Args:
        df (pd.DataFrame): El DataFrame con los datos.
        columna_fecha (str): El nombre de la columna con las fechas.
        fecha_inicio (str): La fecha de inicio para el filtro en formato 'YYYY-MM-DD'.
        fecha_fin (str): La fecha de fin para el filtro en formato 'YYYY-MM-DD'.

    Returns:
        pd.DataFrame: DataFrame resultante de la consulta.
    """
    
    # Asegurarse de que la columna de fechas esté en formato datetime
    #df[columna_fecha] = pd.to_datetime(df[columna_fecha])   ## Si no esta en datetime la columna fecha quitar el #

    # Crear el filtro para el query con BETWEEN
    consulta = f"{columna_fecha} >= '{fecha_inicio}' and {columna_fecha} <= '{fecha_fin}'"
    
    # Aplicar el query con pandas
    resultado = df.query(consulta)

    return resultado

In [14]:
#Estraer Trimestres
#Año 2012
Tr1_2012 = extraer_trimestres_año(Fact_Venta, "Fecha", "2012-01-01", "2012-03-31")
Tr2_2012 = extraer_trimestres_año(Fact_Venta, "Fecha", "2012-04-01", "2012-06-30")
Tr3_2012 = extraer_trimestres_año(Fact_Venta, "Fecha", "2012-07-01", "2012-09-30")
Tr4_2012 = extraer_trimestres_año(Fact_Venta, "Fecha", "2012-10-01", "2012-12-31")

#Año 2013
Tr1_2013 = extraer_trimestres_año(Fact_Venta, "Fecha", "2013-01-01", "2013-03-31")
Tr2_2013 = extraer_trimestres_año(Fact_Venta, "Fecha", "2013-04-01", "2013-06-30")
Tr3_2013 = extraer_trimestres_año(Fact_Venta, "Fecha", "2013-07-01", "2013-09-30")
Tr4_2013 = extraer_trimestres_año(Fact_Venta, "Fecha", "2013-10-01", "2013-12-31")

#Año 2014
Tr1_2014 = extraer_trimestres_año(Fact_Venta, "Fecha", "2014-01-01", "2014-03-31")
Tr2_2014 = extraer_trimestres_año(Fact_Venta, "Fecha", "2014-04-01", "2014-06-30")
Tr3_2014 = extraer_trimestres_año(Fact_Venta, "Fecha", "2014-07-01", "2014-09-30")
Tr4_2014 = extraer_trimestres_año(Fact_Venta, "Fecha", "2014-10-01", "2014-12-31")

In [15]:
#Extraer trimestres
#Año 2015
Tr1_2015 = extraer_trimestres_año(Fact_Venta, "Fecha", "2015-01-01", "2015-03-31")
Tr2_2015 = extraer_trimestres_año(Fact_Venta, "Fecha", "2015-04-01", "2015-06-30")
Tr3_2015 = extraer_trimestres_año(Fact_Venta, "Fecha", "2015-07-01", "2015-09-30")
Tr4_2015 = extraer_trimestres_año(Fact_Venta, "Fecha", "2015-10-01", "2015-12-31")

#Año 2016
Tr1_2016 = extraer_trimestres_año(Fact_Venta, "Fecha", "2016-01-01", "2016-03-31")
Tr2_2016 = extraer_trimestres_año(Fact_Venta, "Fecha", "2016-04-01", "2016-06-30")
Tr3_2016 = extraer_trimestres_año(Fact_Venta, "Fecha", "2016-07-01", "2016-09-30")
Tr4_2016 = extraer_trimestres_año(Fact_Venta, "Fecha", "2016-10-01", "2016-12-31")

#Año 2017
Tr1_2017 = extraer_trimestres_año(Fact_Venta, "Fecha", "2017-01-01", "2017-03-31")
Tr2_2017 = extraer_trimestres_año(Fact_Venta, "Fecha", "2017-04-01", "2017-06-30")
Tr3_2017 = extraer_trimestres_año(Fact_Venta, "Fecha", "2017-07-01", "2017-09-30")
Tr4_2017 = extraer_trimestres_año(Fact_Venta, "Fecha", "2017-10-01", "2017-12-31")

In [16]:
#2012
Tr1_2012.to_csv("Bd/Fact_Venta/2012/T1_2012.csv", index=False) 
Tr2_2012.to_csv("Bd/Fact_Venta/2012/T2_2012.csv", index=False) 
Tr3_2012.to_csv("Bd/Fact_Venta/2012/T3_2012.csv", index=False) 
Tr4_2012.to_csv("Bd/Fact_Venta/2012/T4_2012.csv", index=False) 

#2013
Tr1_2013.to_csv("Bd/Fact_Venta/2013/T1_2013.csv", index=False)
Tr2_2013.to_csv("Bd/Fact_Venta/2013/T2_2013.csv", index=False) 
Tr3_2013.to_csv("Bd/Fact_Venta/2013/T3_2013.csv", index=False) 
Tr4_2013.to_csv("Bd/Fact_Venta/2013/T4_2013.csv", index=False) 

#2012
Tr1_2014.to_csv("Bd/Fact_Venta/2014/T1_2014.csv", index=False) 
Tr2_2014.to_csv("Bd/Fact_Venta/2014/T2_2014.csv", index=False) 
Tr3_2014.to_csv("Bd/Fact_Venta/2014/T3_2014.csv", index=False) 
Tr4_2014.to_csv("Bd/Fact_Venta/2014/T4_2014.csv", index=False) 

In [17]:
#2015
Tr1_2015.to_csv("Bd/Fact_Venta/2015/T1_2015.csv", index=False) 
Tr2_2015.to_csv("Bd/Fact_Venta/2015/T2_2015.csv", index=False) 
Tr3_2015.to_csv("Bd/Fact_Venta/2015/T3_2015.csv", index=False) 
Tr4_2015.to_csv("Bd/Fact_Venta/2015/T4_2015.csv", index=False) 

#2016
Tr1_2016.to_csv("Bd/Fact_Venta/2016/T1_2016.csv", index=False)
Tr2_2016.to_csv("Bd/Fact_Venta/2016/T2_2016.csv", index=False) 
Tr3_2016.to_csv("Bd/Fact_Venta/2016/T3_2016.csv", index=False) 
Tr4_2016.to_csv("Bd/Fact_Venta/2016/T4_2016.csv", index=False) 

#2017
Tr1_2017.to_csv("Bd/Fact_Venta/2017/T1_2017.csv", index=False) 
Tr2_2017.to_csv("Bd/Fact_Venta/2017/T2_2017.csv", index=False) 
Tr3_2017.to_csv("Bd/Fact_Venta/2017/T3_2017.csv", index=False) 
Tr4_2017.to_csv("Bd/Fact_Venta/2017/T4_2017.csv", index=False) 

DIM TIENDA

In [36]:
Dim_Store = df[['Store Number', 'Store Name', 'Address', 'City', 'Zip Code', 'Store Location', 'County']]

In [37]:
Dim_Store.isna().sum()

Store Number          0
Store Name            0
Address            2376
City               2375
Zip Code           2420
Store Location     2375
County            79178
dtype: int64

In [ ]:
#Ver si hay datos para poder rellenar en una columna de ejemplo (no es obligatorio ni mucho menos)
Dim_Store[Dim_Store["Store Number"] == 5250]

,Store Number,Store Name,Address,City,Zip Code,Store Location,County
9039515,5250,East Side Liquor and Tobacco,"3804, Hubbell Ave",Des Moines,50317,"3804, Hubbell Ave\nDes Moines 50317\n(41.63080...",NaN
9039516,5250,East Side Liquor and Tobacco,"3804, Hubbell Ave",Des Moines,50317,"3804, Hubbell Ave\nDes Moines 50317\n(41.63080...",NaN
9039517,5250,East Side Liquor and Tobacco,"3804, Hubbell Ave",Des Moines,50317,"3804, Hubbell Ave\nDes Moines 50317\n(41.63080...",NaN
9039518,5250,East Side Liquor and Tobacco,"3804, Hubbell Ave",Des Moines,50317,"3804, Hubbell Ave\nDes Moines 50317\n(41.63080...",NaN
9039519,5250,East Side Liquor and Tobacco,"3804, Hubbell Ave",Des Moines,50317,"3804, Hubbell Ave\nDes Moines 50317\n(41.63080...",NaN
...,...,...,...,...,...,...,...
12580640,5250,East Side Liquor and Tobacco,3804 Hubbell Ave,Des Moines,50317,3804 Hubbell Ave\nDes Moines 50317\n(41.630805...,POLK
12580641,5250,East Side Liquor and Tobacco,3804 Hubbell Ave,Des Moines,50317,3804 Hubbell Ave\nDes Moines 50317\n(41.630805...,POLK
12580642,5250,East Side Liquor and Tobacco,3804 Hubbell Ave,Des Moines,50317,3804 Hubbell Ave\nDes Moines 50317\n(41.630805...,POLK
12580643,5250,East Side Liquor and Tobacco,3804 Hubbell Ave,Des Moines,50317,3804 Hubbell Ave\nDes Moines 50317\n(41.630805...,POLK


In [28]:
# Funcion para rellenar isna
def fill_column_by_group(dataframe, group_column, fill_column):
    """
    Rellena los valores vacíos en una columna específica usando los valores no vacíos 
    correspondientes al mismo grupo definido por otra columna.

    Args:
        dataframe (pd.DataFrame): El DataFrame original.
        group_column (str): El nombre de la columna para agrupar (por ejemplo, 'Store Number').
        fill_column (str): El nombre de la columna donde se deben rellenar los valores (por ejemplo, 'County').

    Returns:
        pd.DataFrame: DataFrame con los valores de la columna rellenados.
    """
    # Crear un diccionario que mapea el grupo a un valor no vacío en la columna de relleno
    group_to_value = (
        dataframe[dataframe[fill_column].notnull() & (dataframe[fill_column] != '')]
        .groupby(group_column)[fill_column]
        .first()
        .to_dict()
    )
    
    # Usar el diccionario para rellenar los valores vacíos en la columna especificada
    dataframe[fill_column] = dataframe.apply(
        lambda row: group_to_value.get(row[group_column], row[fill_column]) 
        if pd.isnull(row[fill_column]) or row[fill_column] == '' 
        else row[fill_column], 
        axis=1
    )
    return dataframe


FILL COUNTY

In [ ]:
#Rellenar isna 
Dim_Tienda_County = fill_column_by_group(Dim_Store, group_column='Store Number', fill_column='County')

C:\Users\jorge\AppData\Local\Temp\ipykernel_25292\2683391841.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe[fill_column] = dataframe.apply(


In [32]:
#Comprobar si ha rellenado bien
Dim_Tienda_County[Dim_Tienda_County["Store Number"] == 5250]

,Store Number,Store Name,Address,City,Zip Code,Store Location,County
9039515,5250,East Side Liquor and Tobacco,"3804, Hubbell Ave",Des Moines,50317,"3804, Hubbell Ave\nDes Moines 50317\n(41.63080...",POLK
9039516,5250,East Side Liquor and Tobacco,"3804, Hubbell Ave",Des Moines,50317,"3804, Hubbell Ave\nDes Moines 50317\n(41.63080...",POLK
9039517,5250,East Side Liquor and Tobacco,"3804, Hubbell Ave",Des Moines,50317,"3804, Hubbell Ave\nDes Moines 50317\n(41.63080...",POLK
9039518,5250,East Side Liquor and Tobacco,"3804, Hubbell Ave",Des Moines,50317,"3804, Hubbell Ave\nDes Moines 50317\n(41.63080...",POLK
9039519,5250,East Side Liquor and Tobacco,"3804, Hubbell Ave",Des Moines,50317,"3804, Hubbell Ave\nDes Moines 50317\n(41.63080...",POLK
...,...,...,...,...,...,...,...
12580640,5250,East Side Liquor and Tobacco,3804 Hubbell Ave,Des Moines,50317,3804 Hubbell Ave\nDes Moines 50317\n(41.630805...,POLK
12580641,5250,East Side Liquor and Tobacco,3804 Hubbell Ave,Des Moines,50317,3804 Hubbell Ave\nDes Moines 50317\n(41.630805...,POLK
12580642,5250,East Side Liquor and Tobacco,3804 Hubbell Ave,Des Moines,50317,3804 Hubbell Ave\nDes Moines 50317\n(41.630805...,POLK
12580643,5250,East Side Liquor and Tobacco,3804 Hubbell Ave,Des Moines,50317,3804 Hubbell Ave\nDes Moines 50317\n(41.630805...,POLK


In [ ]:
#Comprobar si quedan vacios
Dim_Tienda_County.isna().sum()

Store Number         0
Store Name           0
Address           2376
City              2375
Zip Code          2420
Store Location    2375
County             797
dtype: int64

In [34]:
# Filtrar filas donde la columna 'county' está vacía
filtered_rows = Dim_Tienda_County[Dim_Tienda_County['County'].isnull() | (Dim_Tienda_County['County'] == '')]

# Mostrar únicamente las columnas 'county' y 'store number'
nulls = filtered_rows[['County', 'Store Number']]
nulls.sample(1)

,County,Store Number
9254302,NaN,5220


In [40]:
Dim_Tienda_County[Dim_Tienda_County["Store Number"] == 5220]

,Store Number,Store Name,Address,City,Zip Code,Store Location,County
8777497,5220,Liquor Tobacco & Grocery / Mason Cit,"2771, 4th ST SW",MASON CITY,50401,"2771, 4th ST SW\nMASON CITY 50401\n(43.148281,...",NaN
8777498,5220,Liquor Tobacco & Grocery / Mason Cit,"2771, 4th ST SW",MASON CITY,50401,"2771, 4th ST SW\nMASON CITY 50401\n(43.148281,...",NaN
8777499,5220,Liquor Tobacco & Grocery / Mason Cit,"2771, 4th ST SW",MASON CITY,50401,"2771, 4th ST SW\nMASON CITY 50401\n(43.148281,...",NaN
8777500,5220,Liquor Tobacco & Grocery / Mason Cit,"2771, 4th ST SW",MASON CITY,50401,"2771, 4th ST SW\nMASON CITY 50401\n(43.148281,...",NaN
8777501,5220,Liquor Tobacco & Grocery / Mason Cit,"2771, 4th ST SW",MASON CITY,50401,"2771, 4th ST SW\nMASON CITY 50401\n(43.148281,...",NaN
...,...,...,...,...,...,...,...
9386204,5220,Liquor Tobacco & Grocery / Mason Cit,"2771, 4th St SW",Mason City,50401,"2771, 4th St SW\nMason City 50401\n(43.148281,...",NaN
9386205,5220,Liquor Tobacco & Grocery / Mason Cit,"2771, 4th St SW",Mason City,50401,"2771, 4th St SW\nMason City 50401\n(43.148281,...",NaN
9386206,5220,Liquor Tobacco & Grocery / Mason Cit,"2771, 4th St SW",Mason City,50401,"2771, 4th St SW\nMason City 50401\n(43.148281,...",NaN
9386207,5220,Liquor Tobacco & Grocery / Mason Cit,"2771, 4th St SW",Mason City,50401,"2771, 4th St SW\nMason City 50401\n(43.148281,...",NaN


In [41]:
#Vemos que numero de nan de 5220 coicide con los valores faltantes totales y buscamos el condado por internet y rellenamos
Dim_Tienda_County.loc[Dim_Tienda_County["Store Number"] == 5220, 'County'] = 'Cerro Gordo'

In [42]:
#Comprobar que se relleno correctamente
Dim_Tienda_County[Dim_Tienda_County["Store Number"] == 5220]

,Store Number,Store Name,Address,City,Zip Code,Store Location,County
8777497,5220,Liquor Tobacco & Grocery / Mason Cit,"2771, 4th ST SW",MASON CITY,50401,"2771, 4th ST SW\nMASON CITY 50401\n(43.148281,...",Cerro Gordo
8777498,5220,Liquor Tobacco & Grocery / Mason Cit,"2771, 4th ST SW",MASON CITY,50401,"2771, 4th ST SW\nMASON CITY 50401\n(43.148281,...",Cerro Gordo
8777499,5220,Liquor Tobacco & Grocery / Mason Cit,"2771, 4th ST SW",MASON CITY,50401,"2771, 4th ST SW\nMASON CITY 50401\n(43.148281,...",Cerro Gordo
8777500,5220,Liquor Tobacco & Grocery / Mason Cit,"2771, 4th ST SW",MASON CITY,50401,"2771, 4th ST SW\nMASON CITY 50401\n(43.148281,...",Cerro Gordo
8777501,5220,Liquor Tobacco & Grocery / Mason Cit,"2771, 4th ST SW",MASON CITY,50401,"2771, 4th ST SW\nMASON CITY 50401\n(43.148281,...",Cerro Gordo
...,...,...,...,...,...,...,...
9386204,5220,Liquor Tobacco & Grocery / Mason Cit,"2771, 4th St SW",Mason City,50401,"2771, 4th St SW\nMason City 50401\n(43.148281,...",Cerro Gordo
9386205,5220,Liquor Tobacco & Grocery / Mason Cit,"2771, 4th St SW",Mason City,50401,"2771, 4th St SW\nMason City 50401\n(43.148281,...",Cerro Gordo
9386206,5220,Liquor Tobacco & Grocery / Mason Cit,"2771, 4th St SW",Mason City,50401,"2771, 4th St SW\nMason City 50401\n(43.148281,...",Cerro Gordo
9386207,5220,Liquor Tobacco & Grocery / Mason Cit,"2771, 4th St SW",Mason City,50401,"2771, 4th St SW\nMason City 50401\n(43.148281,...",Cerro Gordo


FILL CITY

In [43]:
# Filtrar filas donde la columna 'city' está vacía
filtered_rows = Dim_Store[Dim_Store['City'].isnull() | (Dim_Store['City'] == '')]

# Mostrar únicamente las columnas 'city' y 'store number'
nulls = filtered_rows[['City', 'Store Number']]
nulls

,City,Store Number
9906975,NaN,4725
9907028,NaN,4725
9907046,NaN,4725
9907064,NaN,4767
9907139,NaN,4725
...,...,...
12498215,NaN,4378
12498216,NaN,4378
12498217,NaN,4378
12498218,NaN,4378


In [44]:
Dim_Store[Dim_Store["Store Number"] == 4725].sample(1)

,Store Number,Store Name,Address,City,Zip Code,Store Location,County
12563027,4725,Casey's General Store #1548 / Ankeny,2905 W 1st St,Ankeny,50023,"2905 W 1st St\nAnkeny 50023\n(41.731708, -93.6...",Polk


In [45]:
Dim_Tienda_City = fill_column_by_group(Dim_Tienda_County, group_column='Store Number', fill_column='City')

C:\Users\jorge\AppData\Local\Temp\ipykernel_25292\2683391841.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe[fill_column] = dataframe.apply(


In [46]:
Dim_Tienda_City.isna().sum()

Store Number         0
Store Name           0
Address           2376
City                 0
Zip Code          2420
Store Location    2375
County               0
dtype: int64

In [47]:
Dim_Tienda_City[Dim_Tienda_City["Store Number"] == 4725].sample(1)

,Store Number,Store Name,Address,City,Zip Code,Store Location,County
7571001,4725,Casey's General Store #1548 / Ankeny,"2905, W 1ST ST",ANKENY,50023,"2905, W 1ST ST\nANKENY 50023\n(41.731704, -93....",Polk


FILL ADRESS

In [48]:
Dim_Tienda_Address = fill_column_by_group(Dim_Tienda_City, group_column='Store Number', fill_column='Address')

C:\Users\jorge\AppData\Local\Temp\ipykernel_25292\2683391841.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe[fill_column] = dataframe.apply(


In [49]:
Dim_Tienda_Address.isna().sum()

Store Number         0
Store Name           0
Address              1
City                 0
Zip Code          2420
Store Location    2375
County               0
dtype: int64

In [166]:
Dim_Tienda_Address[Dim_Tienda_Address["Address"].isnull()]

,Store Number,Store Name,Address,City,Zip Code,Store Location,County
11757989,9932,"3-Oaks Distillery, LLC",NaN,Dubuque,52002.0,"Dubuque 52002\n(42.516027, -90.766335)",DUBUQUE


In [51]:
#Buscamos por internet y rellenamos
Dim_Tienda_Address.loc[Dim_Tienda_Address["Store Number"] == 9932, 'Address'] = '23039 Pfeiler Rd, Holy Cross'

FILL ZIP CODE

In [52]:
Dim_Tienda_Zip = fill_column_by_group(Dim_Tienda_Address, group_column='Store Number', fill_column='Zip Code')

C:\Users\jorge\AppData\Local\Temp\ipykernel_25292\2683391841.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe[fill_column] = dataframe.apply(


In [53]:
Dim_Tienda_Zip.isna().sum()

Store Number         0
Store Name           0
Address              0
City                 0
Zip Code             0
Store Location    2375
County               0
dtype: int64

FILL STORE LOCATION

In [66]:
#Nos damos cuenta de que Store Location tiene mas de 1 valor unico por columna y no podemos extraer todavia latitud y longitud
value_counts = Dim_Tienda_Zip[Dim_Tienda_Zip["Store Number"] == 4255]["Store Location"].value_counts()
print(value_counts)

Store Location
512 8TH SE\nORANGE CITY 51041\n(42.997487, -96.052388)    6473
512 8th SE\nOrange City 51041\n(42.997486, -96.05236)     1319
512 8th SE\nOrange City 51041\n                           1063
512 8TH SE\nORANGE CITY 51041\n                            466
Name: count, dtype: int64


In [72]:
def rellenar_con_valor_mas_frecuente(df, columna_clave, columna_objetivo, condicion):
    """
    Rellena los valores de la columna de destino usando el valor más frecuente que cumpla una condición,
    basado en la columna clave.

    Args:
        df (pd.DataFrame): El DataFrame original.
        columna_clave (str): Nombre de la columna clave para agrupar.
        columna_objetivo (str): Nombre de la columna que se rellenará.
        condicion (func): Función que define la condición para incluir valores en el cálculo del más frecuente.

    Returns:
        pd.DataFrame: DataFrame con la columna de destino rellenada.
    """
    # Filtrar la columna de destino según la condición
    filtered_data = df[df[columna_objetivo].notnull()]
    filtered_data = filtered_data[filtered_data[columna_objetivo].apply(condicion)]

    # Crear un diccionario con el valor más frecuente que cumple la condición
    valores_mas_frec = (
        filtered_data.groupby(columna_clave)[columna_objetivo]
        .agg(lambda x: x.value_counts().idxmax())  # Encuentra el valor más frecuente
        .to_dict()
    )
    
    # Usar el diccionario para rellenar los valores en la columna de destino
    df[columna_objetivo] = df.apply(
        lambda row: valores_mas_frec.get(row[columna_clave], row[columna_objetivo]),
        axis=1
    )
    
    return df


In [75]:
# Aplicar la función al DataFrame
Dim_Tienda = rellenar_con_valor_mas_frecuente(
    df= Dim_Tienda_Zip,
    columna_clave="Store Number",
    columna_objetivo="Store Location",
    condicion=lambda x: ',' in x  # Condición: Contiene una coma
)

C:\Users\jorge\AppData\Local\Temp\ipykernel_25292\4107278958.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[columna_objetivo] = df.apply(


In [76]:
Dim_Tienda.isna().sum()

Store Number      0
Store Name        0
Address           0
City              0
Zip Code          0
Store Location    0
County            0
dtype: int64

LIMPIEZA DATOS

In [84]:
#Eliminar texto despues de "/"
Dim_Tienda['Store Name'] = Dim_Tienda['Store Name'].str.replace(r'\/.*', '', regex=True)

C:\Users\jorge\AppData\Local\Temp\ipykernel_25292\1848342622.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Dim_Tienda['Store Name'] = Dim_Tienda['Store Name'].str.replace(r'\/.*', '', regex=True)


In [90]:
Dim_Tienda.drop(['Nombre Tienda', 'Store Location'], axis=1, inplace=True)

C:\Users\jorge\AppData\Local\Temp\ipykernel_25292\3079979653.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Dim_Tienda.drop(['Nombre Tienda', 'Store Location'], axis=1, inplace=True)


In [ ]:
#Eliminar duplicados
Dim_Tienda.drop_duplicates(inplace=True)

In [ ]:
# Capitalizar todas las columnas de texto
Dim_Tienda = Dim_Tienda.applymap(lambda x: x.capitalize() if isinstance(x, str) else x)

In [92]:
Dim_Tienda.rename(columns={'Store Number': 'Id Tienda', 'Address': 'Direccion', "City": "Ciudad", "Zip Code": "Codigo Postal", "County": "Condado", "Store Name": "Nombre Tienda"}, inplace=True)

C:\Users\jorge\AppData\Local\Temp\ipykernel_25292\2664090332.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Dim_Tienda.rename(columns={'Store Number': 'Id Tienda', 'Address': 'Direccion', "City": "Ciudad", "Zip Code": "Codigo Postal", "County": "Condado", "Store Name": "Nombre Tienda"}, inplace=True)


In [98]:
Dim_Tienda.drop_duplicates(inplace=True)

C:\Users\jorge\AppData\Local\Temp\ipykernel_25292\2301584826.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Dim_Tienda.drop_duplicates(inplace=True)


In [99]:
Dim_Tienda.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8092 entries, 0 to 12588601
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Id Tienda      8092 non-null   int64 
 1   Nombre Tienda  8092 non-null   object
 2   Direccion      8092 non-null   object
 3   Ciudad         8092 non-null   object
 4   Codigo Postal  8092 non-null   object
 5   Condado        8092 non-null   object
dtypes: int64(1), object(5)
memory usage: 442.5+ KB


EXPORTAR

In [100]:
#Estraer df
Dim_Tienda.to_csv("Bd/Dim/Dim_Tienda.csv", index=False)  # index=False para no guardar el índice como columna

DIM ARTICULO

In [19]:
df.columns

Index(['Invoice/Item Number', 'Date', 'Store Number', 'Store Name', 'Address',
       'City', 'Zip Code', 'Store Location', 'County Number', 'County',
       'Category', 'Category Name', 'Vendor Number', 'Vendor Name',
       'Item Number', 'Item Description', 'Pack', 'Bottle Volume (ml)',
       'State Bottle Cost', 'State Bottle Retail', 'Bottles Sold',
       'Sale (Dollars)', 'Volume Sold (Liters)', 'Volume Sold (Gallons)'],
      dtype='object')

In [20]:
Dim_Articulo_1 = df[['Item Number', 'Item Description','Category Name', 'Vendor Name', 'Pack', 'Bottle Volume (ml)','State Bottle Cost', 'State Bottle Retail']]

In [21]:
Dim_Articulo_1.isna().sum()

Item Number                0
Item Description           0
Category Name          16086
Vendor Name                1
Pack                       0
Bottle Volume (ml)         0
State Bottle Cost         10
State Bottle Retail       10
dtype: int64

In [24]:
# Filtrar filas donde la columna 'county' está vacía
filtered_rows = Dim_Articulo_1[Dim_Articulo_1['Category Name'].isnull() | (Dim_Articulo_1['Category Name'] == '')]

# Mostrar únicamente las columnas 'county' y 'store number'
nulls = filtered_rows[['Category Name', 'Item Number']]
nulls

,Category Name,Item Number
0,NaN,297
1,NaN,297
2,NaN,249
3,NaN,237
4,NaN,249
...,...,...
12588346,NaN,36122
12589217,NaN,80145
12589284,NaN,43439
12589614,NaN,100041


In [26]:
values = Dim_Articulo_1[Dim_Articulo_1["Item Number"] == 43439].value_counts()
values

Item Number  Item Description                  Category Name  Vendor Name      Pack  Bottle Volume (ml)  State Bottle Cost  State Bottle Retail
43439        Captain Morgan Jack-O Blast Mini  Flavored Rum   DIAGEO AMERICAS  12    500                 $4.96              $7.44                  210
Name: count, dtype: int64

In [29]:
Dim_Articulo_2 = fill_column_by_group(Dim_Articulo_1, group_column='Item Number', fill_column='Category Name')

C:\Users\jorge\AppData\Local\Temp\ipykernel_17544\3984453287.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe[fill_column] = dataframe.apply(


In [30]:
Dim_Articulo_2.isna().sum()

Item Number               0
Item Description          0
Category Name          1488
Vendor Name               1
Pack                      0
Bottle Volume (ml)        0
State Bottle Cost        10
State Bottle Retail      10
dtype: int64

In [33]:
# Filtrar filas donde la columna 'county' está vacía
filtered_rows = Dim_Articulo_2[Dim_Articulo_2['State Bottle Cost'].isnull() | (Dim_Articulo_2['State Bottle Cost'] == '')]

# Mostrar únicamente las columnas 'county' y 'store number'
nulls = filtered_rows[['State Bottle Cost', 'Item Number']]
nulls

,State Bottle Cost,Item Number
199088,NaN,38180
356024,NaN,38180
1247099,NaN,35920
1306628,NaN,35920
1428507,NaN,38180
1896582,NaN,38180
3003583,NaN,38180
4337985,NaN,35920
4536650,NaN,38180
6017165,NaN,38180


In [34]:
Dim_Articulo_3 = fill_column_by_group(Dim_Articulo_2, group_column='Item Number', fill_column='State Bottle Cost')
Dim_Articulo_4 = fill_column_by_group(Dim_Articulo_3, group_column='Item Number', fill_column='State Bottle Retail')

C:\Users\jorge\AppData\Local\Temp\ipykernel_17544\3984453287.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe[fill_column] = dataframe.apply(


In [40]:
filtered_rows = Dim_Articulo_4[Dim_Articulo_4['Vendor Name'].isnull()]
filtered_rows

,Item Number,Item Description,Category Name,Vendor Name,Pack,Bottle Volume (ml),State Bottle Cost,State Bottle Retail
11170669,905433,Laphroaig Quarter Cask,NaN,NaN,6,750,$4.17,$47.24


In [39]:
Dim_Articulo_4[Dim_Articulo_4["Item Number"] == 905433] 

,Item Number,Item Description,Category Name,Vendor Name,Pack,Bottle Volume (ml),State Bottle Cost,State Bottle Retail
11170669,905433,Laphroaig Quarter Cask,NaN,NaN,6,750,$4.17,$47.24


In [46]:
value = Dim_Articulo_4[(Dim_Articulo_4["Item Description"] == "Laphroaig Quarter Cask") & (Dim_Articulo_4["Bottle Volume (ml)"] == 750)]
value["Category Name"].value_counts()

Category Name
SCOTCH WHISKIES       190
SINGLE MALT SCOTCH     25
Scotch Whiskies         9
Name: count, dtype: int64

In [47]:
Dim_Articulo_4.loc[Dim_Articulo_4['Item Number'] == 905433, 'Vendor Name'] = "Jim Beam Brands"
Dim_Articulo_4.loc[Dim_Articulo_4['Item Number'] == 905433, 'Category Name'] = "Scotch Whiskies"

In [48]:
Dim_Articulo_4.isna().sum()

Item Number               0
Item Description          0
Category Name          1487
Vendor Name               0
Pack                      0
Bottle Volume (ml)        0
State Bottle Cost         0
State Bottle Retail       0
dtype: int64

In [57]:
def fill_column_by_multiple_groups(dataframe, group_columns, fill_column):
    """
    Rellena los valores vacíos en una columna específica usando los valores no vacíos 
    correspondientes al mismo grupo definido por una o más columnas.

    Args:
        dataframe (pd.DataFrame): El DataFrame original.
        group_columns (list): Lista de nombres de columnas para agrupar (por ejemplo, ['Store Number', 'City']).
        fill_column (str): El nombre de la columna donde se deben rellenar los valores (por ejemplo, 'County').

    Returns:
        pd.DataFrame: DataFrame con los valores de la columna rellenados.
    """
    # Crear un diccionario que mapea los grupos a un valor no vacío en la columna de relleno
    group_to_value = (
        dataframe[dataframe[fill_column].notnull() & (dataframe[fill_column] != '')]
        .groupby(group_columns)[fill_column]
        .first()
        .to_dict()
    )
    
    # Usar el diccionario para rellenar los valores vacíos en la columna especificada
    dataframe[fill_column] = dataframe.apply(
        lambda row: group_to_value.get(
            tuple(row[col] for col in group_columns),  # Convertir las columnas de grupo en tupla como clave
            row[fill_column]
        ) if pd.isnull(row[fill_column]) or row[fill_column] == '' else row[fill_column], 
        axis=1
    )
    return dataframe

In [58]:
Dim_Articulo_5 = fill_column_by_multiple_groups(Dim_Articulo_4, ["Vendor Name", "State Bottle Cost", "State Bottle Retail", "Item Description"], "Category Name")

C:\Users\jorge\AppData\Local\Temp\ipykernel_17544\1137355349.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe[fill_column] = dataframe.apply(


In [59]:
Dim_Articulo_5.isna().sum()

Item Number               0
Item Description          0
Category Name          1432
Vendor Name               0
Pack                      0
Bottle Volume (ml)        0
State Bottle Cost         0
State Bottle Retail       0
dtype: int64

In [64]:
filtered_rows = Dim_Articulo_5[Dim_Articulo_4['Category Name'].isnull()]

In [65]:
filtered_rows["Item Number"].value_counts().head(10)

Item Number
3337      201
100229    138
559        90
1488       86
100232     77
100282     68
80128      67
905211     41
333        40
1475       39
Name: count, dtype: int64

In [83]:
Dim_Articulo_5[Dim_Articulo_5["Item Number"] == 80128] 

,Item Number,Item Description,Category Name,Vendor Name,Pack,Bottle Volume (ml),State Bottle Cost,State Bottle Retail
12327523,80128,Ezra Brooks Bourbon Cream Mini,NaN,LUXCO INC,12,50,$4.67,$7.01
12330430,80128,Ezra Brooks Bourbon Cream Mini,NaN,LUXCO INC,12,50,$4.67,$7.01
12331666,80128,Ezra Brooks Bourbon Cream Mini,NaN,LUXCO INC,12,50,$4.67,$7.01
12332670,80128,Ezra Brooks Bourbon Cream Mini,NaN,LUXCO INC,12,50,$4.67,$7.01
12333433,80128,Ezra Brooks Bourbon Cream Mini,NaN,LUXCO INC,12,50,$4.67,$7.01
...,...,...,...,...,...,...,...,...
12539927,80128,Ezra Brooks Bourbon Cream Mini,NaN,LUXCO INC,12,50,$4.67,$7.01
12556907,80128,Ezra Brooks Bourbon Cream Mini,NaN,LUXCO INC,12,50,$4.67,$7.01
12574854,80128,Ezra Brooks Bourbon Cream Mini,NaN,LUXCO INC,12,50,$4.67,$7.01
12585804,80128,Ezra Brooks Bourbon Cream Mini,NaN,LUXCO INC,12,50,$4.67,$7.01


In [85]:
Dim_Articulo_5.loc[Dim_Articulo_5["Item Number"] == 3337, 'Category Name'] = 'Bourbon Whiskey'
Dim_Articulo_5.loc[Dim_Articulo_5["Item Number"] == 100229, 'Category Name'] = 'Whiskey'
Dim_Articulo_5.loc[Dim_Articulo_5["Item Number"] == 559, 'Category Name'] = 'Vodka'
Dim_Articulo_5.loc[Dim_Articulo_5["Item Number"] == 1488, 'Category Name'] = 'Vermouth'
Dim_Articulo_5.loc[Dim_Articulo_5["Item Number"] == 100232, 'Category Name'] = 'Tequila'
Dim_Articulo_5.loc[Dim_Articulo_5["Item Number"] == 100282, 'Category Name'] = 'Whiskey'
Dim_Articulo_5.loc[Dim_Articulo_5["Item Number"] == 80128, 'Category Name'] = 'Liqueur'

In [87]:
Dim_Articulo_5["Category Name"].fillna("Desconocido", inplace=True)

C:\Users\jorge\AppData\Local\Temp\ipykernel_17544\958807941.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Dim_Articulo_5["Category Name"].fillna("Desconocido", inplace=True)


In [88]:
Dim_Articulo_5.isna().sum()

Item Number            0
Item Description       0
Category Name          0
Vendor Name            0
Pack                   0
Bottle Volume (ml)     0
State Bottle Cost      0
State Bottle Retail    0
dtype: int64

In [89]:
Dim_Articulo_5.rename(columns={"Item Number":"Id Articulo", "Item Description": "Nombre Articulo", "Category Name": "Categoria", "Vendor Name": "Nombre Vendedor", "Bottle Volume (ml)": "Tamaño (ml)", "State Bottle Cost": "Precio Coste", "State Bottle Retail": "Precio Venta"}, inplace=True)

C:\Users\jorge\AppData\Local\Temp\ipykernel_17544\4214513777.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Dim_Articulo_5.rename(columns={"Item Number":"Id Articulo", "Item Descripcion": "Nombre Articulo", "Category Name": "Categoria", "Vendor Name": "Nombre Vendedor", "Bottle Volume (ml)": "Tamaño (ml)", "State Bottle Cost": "Precio Coste", "State Bottle Retail": "Precio Venta"}, inplace=True)


In [91]:
Dim_Articulo_5.rename(columns={"Item Description": "Nombre Articulo"}, inplace=True)

C:\Users\jorge\AppData\Local\Temp\ipykernel_17544\42809959.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Dim_Articulo_5.rename(columns={"Item Description": "Nombre Articulo"}, inplace=True)


In [92]:
Dim_Articulo_5.head(1)

,Id Articulo,Nombre Articulo,Categoria,Nombre Vendedor,Pack,Tamaño (ml),Precio Coste,Precio Venta
0,297,Templeton Rye w/Flask,DECANTERS & SPECIALTY PACKAGES,Wilson Daniels Ltd.,6,750,$18.09,$27.14


In [93]:
Dim_Articulo_5.drop_duplicates(inplace=True)

C:\Users\jorge\AppData\Local\Temp\ipykernel_17544\360612538.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Dim_Articulo_5.drop_duplicates(inplace=True)


In [94]:
Dim_Articulo_5.info()

<class 'pandas.core.frame.DataFrame'>
Index: 17298 entries, 0 to 12590343
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Id Articulo      17298 non-null  int64 
 1   Nombre Articulo  17298 non-null  object
 2   Categoria        17298 non-null  object
 3   Nombre Vendedor  17298 non-null  object
 4   Pack             17298 non-null  int64 
 5   Tamaño (ml)      17298 non-null  int64 
 6   Precio Coste     17298 non-null  object
 7   Precio Venta     17298 non-null  object
dtypes: int64(3), object(5)
memory usage: 1.2+ MB


In [95]:
#Estraer df
Dim_Articulo_5.to_csv("Bd/Dim/Dim_Articulo.csv", index=False)  # index=False para no guardar el índice como columna